In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# ------- Import Libs ---------
from datetime import datetime
import os
import pathlib

import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import math
import pytz

import warnings
import re

import plotly.express as px

alt.renderers.enable("altair_viewer")

# Disbale the max row limit for altair datasets.
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
# Set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

print(CWD)

/Users/zakwatts/Coding/OCF/pv-site-prediction


In [6]:
# This function does the same as above but orders the dataframe correct
# I should also make the folder path a variable so that I am perform some quick stats analysis on it
def folder_data_load_sorted(folder_path):
    # This is a list of all of the filenames
    files = os.listdir(folder_path)

    # Read all Excel files into a list of dataframes
    dataframes = []

    for filename in files:
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            print(file_path)

            # Extract the reference date from the filename
            reference_date = filename.split("_")[0]

            df = pd.read_excel(file_path, engine="openpyxl")

            # create new columns in the data for date specific
            df["Year"] = reference_date.split("-")[0]
            df["Month"] = reference_date.split("-")[1]
            df["Day"] = reference_date.split("-")[2]

            # Convert the 'Time' column to datetime.time objects
            df["Time2"] = pd.to_datetime(df["Time"], format="%H:%M:%S").dt.time
            df["Time2"] = df["Time2"].astype(str)

            df["Date"] = pd.to_datetime(df["Year"] + df["Month"] + df["Day"], format="%Y-%m-%d")

            # Convert the 'Time' column to datetime objects using the extracted reference date
            df["Datetime"] = pd.to_datetime(
                reference_date + " " + df["Time2"], format="%Y-%m-%d %H:%M:%S"
            )

            # This returns a tuple
            dataframes.append((reference_date, df))

    dataframes.sort(key=lambda x: x[0])

    sorted_dataframes = [df for _, df in dataframes]

    return sorted_dataframes

In [7]:
# Best to create a way to look at all of the data now rather then just one specific year.
def load_data_from_all_years(parent_folder_path):
    # Initialize an empty list to store the dataframes
    all_dataframes = []

    # Loop over each year's folder and call the folder_data_load_sorted function
    for year in range(2018, 2024):
        folder_path = os.path.join(parent_folder_path, str(year))
        dataframes = folder_data_load_sorted(folder_path)
        all_dataframes.extend(dataframes)

    return all_dataframes

In [8]:
# A function to load a single excel file
def read_excel_file(file_path):
    df = pd.read_excel(file_path, engine="openpyxl")
    return df

In [9]:
# ---------- 15 min data ------------
def load_all_15min(parent_folder_path):

    all_dataframes = load_data_from_all_years(parent_folder_path)
    all_combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Sort the DataFrame based on the 'Datetime' column
    all_combined_df_sort = all_combined_df.sort_values(by="Datetime")

    return all_combined_df_sort

In [10]:
# define the function to transform the dates to fix the bug
def transform_date(date_str):
    date_format = "%Y-%m-%d %H:%M:%S"
    date_obj = datetime.strptime(date_str, date_format)
    new_date = date_obj.replace(hour=date_obj.minute, minute=date_obj.second, second=0)
    return new_date.strftime(date_format)

In [32]:
# --------- 15 min data opperation ----------
# 1. Load the data into a single list
parent_folder_path = "./data/island_A/clean_data/15-min-PV/"
data_15min = load_all_15min(parent_folder_path)

print("-------- COMPLETE 1 -------")

# This code here is if you want to just load a single year of 15 min data
#     folder_path = "./data/island_A/15-min-PV/2019/"
#     data_15min = folder_data_load_sorted(folder_path)

./data/island_A/clean_data/15-min-PV/2018/2018-12-20_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-18_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-27_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-11_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-16_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-29_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-03_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-04_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-19_PVMalta.xlsx


/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


./data/island_A/clean_data/15-min-PV/2018/2018-12-26_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-21_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-05_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-02_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-17_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-28_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-10_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-25_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-22_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-08_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-30_PVMalta.xlsx


/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


./data/island_A/clean_data/15-min-PV/2018/2018-11-06_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-01_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-14_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-13_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-09_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-23_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-24_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-12_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-15_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-07_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-26_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-19_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-21_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-12-28_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2018/2018-11-17_PVMalta.xlsx
./data/isl

...

./data/island_A/clean_data/15-min-PV/2023/2023-03-14_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-03-13_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-01-05_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-01-02_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-03-06_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-03-01_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-01-08_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-25_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-22_PVMalta.xlsx


/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/z

./data/island_A/clean_data/15-min-PV/2023/2023-02-14_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-13_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-06_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-01_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-23_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-24_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-01-09_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-07_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-12_PVMalta.xlsx
./data/island_A/clean_data/15-min-PV/2023/2023-02-15_PVMalta.xlsx


/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/zakwatts/opt/anaconda3/envs/OCF/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/z

-------- COMPLETE 1 -------


In [33]:
# 2. Fix the bugs with datetime interp
# define the bug_start and bug_end dates + May need to enter new dates and check format
bug_start = datetime.strptime("2018-12-19 23:45:00", "%Y-%m-%d %H:%M:%S")
bug_end = datetime.strptime("2019-10-05 00:00:00", "%Y-%m-%d %H:%M:%S")

data_15min["Datetime"] = pd.to_datetime(data_15min["Datetime"], format="%Y-%m-%d %H:%M:%S")

# apply the transformation to the appropriate rows
mask = (data_15min["Datetime"] > bug_start) & (data_15min["Datetime"] < bug_end)
data_15min.loc[mask, "Datetime"] = data_15min.loc[mask, "Datetime"].apply(
    lambda x: transform_date(x.strftime("%Y-%m-%d %H:%M:%S"))
)
print("-------- COMPLETE 2 -------")



print("-------- COMPLETE 5 -------")

-------- COMPLETE 2 -------
-------- COMPLETE 5 -------


In [34]:
# 5. Save the data as a csv
data_15min.to_csv("data/island_A/exported/data_15min_clean.csv", index=False)

data_15min_conv = data_15min
print("-------- COMPLETE 6 -------")

-------- COMPLETE 6 -------


In [41]:
data_15min.head(5)

,Time,15-Minute Output MWh,Power at point in time MW,Number of meters utilised,Total Installed Capacity MWp,Year,Month,Day,Time2,Date,Hour number,Hourly Average,datetimeUTC
Datetime,,,,,,,,,,,,,
2018-11-01 00:00:00,00:00:00,0.0,0.0,0.0,127.451398,2018,11,01,00:00:00,2018-11-01,NaN,NaN,2018-10-31 23:00:00+00:00
2018-11-01 00:15:00,00:15:00,0.0,0.0,0.0,127.432000,2018,11,01,00:15:00,2018-11-01,NaN,NaN,2018-10-31 23:15:00+00:00
2018-11-01 00:30:00,00:30:00,0.0,0.0,24.0,127.432000,2018,11,01,00:30:00,2018-11-01,NaN,NaN,2018-10-31 23:30:00+00:00
2018-11-01 00:45:00,00:45:00,0.0,0.0,21.0,127.432000,2018,11,01,00:45:00,2018-11-01,NaN,NaN,2018-10-31 23:45:00+00:00
2018-11-01 01:00:00,01:00:00,0.0,0.0,22.0,127.432000,2018,11,01,01:00:00,2018-11-01,NaN,NaN,2018-11-01 00:00:00+00:00


In [45]:
# 3. Apply UTC Convention
# View all timezones using the code below
"""
>>> import pytz
>>> pytz.all_timezones
"""

def convert_to_utc(df, source_timezone):
    # Ensure the DataFrame has a DatetimeIndex
#     if not isinstance(df.index, pd.DatetimeIndex):
#         raise ValueError("The DataFrame must have a DatetimeIndex.")

    # Create timezone objects for source and target (UTC) timezones
    source_tz = pytz.timezone(source_timezone)
    target_tz = pytz.UTC
    
    
    # Convert the "datetime" column to a DatetimeIndex
    #df['Datetime'] = pd.to_datetime(df['Datetime'])
    
    
    if not isinstance(df.index, pd.DatetimeIndex):
        # Convert the "datetime" column to a DatetimeIndex
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        df.set_index('Datetime', inplace=True)

    #df.set_index('Datetime', inplace=True)

    
    # Localize the DatetimeIndex to the source timezone, handling ambiguous and non-existent times
    df_source_tz = df.index.tz_localize(source_tz, ambiguous='NaT', nonexistent='NaT')
    
    # Convert the DatetimeIndex to the target timezone (UTC)
    df_utc = df_source_tz.tz_convert(target_tz)
    
    # Set the DatetimeIndex as a column in the DataFrame
    df['datetimeUTC'] = df_utc
    
    
    
    df.set_index('datetimeUTC', inplace=True)
    
    # to get rid of the time zone different impliment:
    # Format the DatetimeIndex without the timezone offset
    # df_utc.index = df_utc.index.strftime('%Y-%m-%d %H:%M:%S')
    
    return df

# orginally got this error: NonExistentTimeError: 2019-03-31 02:00:00, so they dont adjust for daylight savings.


source_timezone = 'Europe/Malta'  # Replace with the desired timezone
data_15min_utc = convert_to_utc(data_15min_conv, source_timezone)


# will need to createa a lamda/function to go through all of the 15min data to convert it
# to the correct UTC. It would be worth making this into a function so it can be used by the hourly.
# Want it so that i just have to pass a dataframe that has a Datetime funciton into it.

print("-------- COMPLETE 3 -------")



TypeError: Already tz-aware, use tz_convert to convert.

In [46]:
data_15min_utc

,Time,15-Minute Output MWh,Power at point in time MW,Number of meters utilised,Total Installed Capacity MWp,Year,Month,Day,Time2,Date,Hour number,Hourly Average
datetimeUTC,,,,,,,,,,,,
2018-10-31 23:00:00+00:00,00:00:00,0.0,0.0,0.0,127.451398,2018,11,01,00:00:00,2018-11-01,NaN,NaN
2018-10-31 23:15:00+00:00,00:15:00,0.0,0.0,0.0,127.432000,2018,11,01,00:15:00,2018-11-01,NaN,NaN
2018-10-31 23:30:00+00:00,00:30:00,0.0,0.0,24.0,127.432000,2018,11,01,00:30:00,2018-11-01,NaN,NaN
2018-10-31 23:45:00+00:00,00:45:00,0.0,0.0,21.0,127.432000,2018,11,01,00:45:00,2018-11-01,NaN,NaN
2018-11-01 00:00:00+00:00,01:00:00,0.0,0.0,22.0,127.432000,2018,11,01,01:00:00,2018-11-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-17 21:45:00+00:00,22:45:00,0.0,0.0,0.0,223.267406,2023,03,17,22:45:00,2023-03-17,NaN,NaN
2023-03-17 22:00:00+00:00,23:00:00,0.0,0.0,0.0,223.267406,2023,03,17,23:00:00,2023-03-17,23.0,0.0
2023-03-17 22:15:00+00:00,23:15:00,0.0,0.0,0.0,223.267406,2023,03,17,23:15:00,2023-03-17,NaN,NaN


In [49]:
def drop_unused_15min(df):
    
    df = df.drop(['Time','Year','Month','Day','Time2','Date', 'Hour number', 'Hourly Average'],1)
        
    return df

data_15min_utc_drop = drop_unused_15min(data_15min_utc)


/var/folders/tk/fz0371jn0z5b2j4kj_ycxyh00000gn/T/ipykernel_24285/1432555520.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['Time','Year','Month','Day','Time2','Date', 'Hour number', 'Hourly Average'],1)


In [50]:
data_15min_utc_drop

,15-Minute Output MWh,Power at point in time MW,Number of meters utilised,Total Installed Capacity MWp
datetimeUTC,,,,
2018-10-31 23:00:00+00:00,0.0,0.0,0.0,127.451398
2018-10-31 23:15:00+00:00,0.0,0.0,0.0,127.432000
2018-10-31 23:30:00+00:00,0.0,0.0,24.0,127.432000
2018-10-31 23:45:00+00:00,0.0,0.0,21.0,127.432000
2018-11-01 00:00:00+00:00,0.0,0.0,22.0,127.432000
...,...,...,...,...
2023-03-17 21:45:00+00:00,0.0,0.0,0.0,223.267406
2023-03-17 22:00:00+00:00,0.0,0.0,0.0,223.267406
2023-03-17 22:15:00+00:00,0.0,0.0,0.0,223.267406


In [83]:
# 8. Convert into Xarray and then NetCDF format
# Converting to an Xarray
def pdtocdf(df, file_name):
    
    data_array = xr.Dataset(df)
    
    save_directory = "./data/island_A/exported/"
    file_end = ".nc"
    full_file_path = save_directory + file_name + file_end
    
    # Save the DataArray as a NetCDF file
    data_array.to_netcdf(full_file_path)
    
    print(f"File saved at: {full_file_path}")
    
    return data_array
    
data_15min_xarray = pdtocdf(data_15min_utc_drop, "data_15min_FIN")

File saved at: ./data/island_A/exported/data_15min_FIN.nc


In [57]:
data_15min_xarray

<xarray.Dataset>
Dimensions:                       (datetimeUTC: 153408)
Coordinates:
  * datetimeUTC                   (datetimeUTC) object 1541026800000000000 .....
Data variables:
    15-Minute Output MWh          (datetimeUTC) float64 0.0 0.0 0.0 ... 0.0 0.0
    Power at point in time MW     (datetimeUTC) float64 0.0 0.0 0.0 ... 0.0 0.0
    Number of meters utilised     (datetimeUTC) float64 0.0 0.0 24.0 ... 0.0 0.0
    Total Installed Capacity MWp  (datetimeUTC) float64 127.5 127.4 ... 223.3

In [ ]:
# 6. Create a new rolling sum of the 15min data
data_15min_resample = data_15min

# data_15min_rolling['15-Minute Output MWh'].rolling(4).sum()
# data_15min_rolling['Power at point in time MW'].rolling(4).sum()


# Assuming your DataFrame has a 'timestamp' column and columns 'A' and 'B'
data_15min_resample["Datetime"] = pd.to_datetime(data_15min_resample["Datetime"])
data_15min_resample.set_index("Datetime", inplace=True)

# Resample column 'B' to hourly frequency and sum the values
df_B_hourly_sum_MWh = data_15min_resample["15-Minute Output MWh"].resample("H").sum()
df_B_hourly_sum_MW = data_15min_resample["Power at point in time MW"].resample("H").sum()

In [ ]:
# 7. Save the data as a csv
df_B_hourly_sum_MWh.to_csv("data/island_A/exported/15mintohour_MWh.csv", index=True)

# 7. Save the data as a csv
df_B_hourly_sum_MW.to_csv("data/island_A/exported/15mintohour_MW.csv", index=True)

print("-------- COMPLETE 7 -------")

In [ ]:
# 7.5 Complete the normalisation.

In [ ]:
# 7. Save the data as a csv
data_15min_rolling.to_csv("data/island_A/exported/data_15min_rolling.csv", index=False)
print("-------- COMPLETE 8 -------")

In [58]:
# ---------- Hourly data -------------
# 1. Load the data
folder_path = "./data/island_A/Hourly-PV/format_test/HourlyPVgeneratedUnits_2018_test.xlsx"
# folder_path = './data/island_A/15-min-PV/2019/2019-01-03_PVMalta.xlsx'
hourly_data_raw = read_excel_file(folder_path)

print("--------- COMPLETE 1 --------")

--------- COMPLETE 1 --------


In [65]:
# 2. Convert into usable format (Transpose of hours)
def transpose_data(df):

    # Convert column names to integers
    hour_columns = [col for col in df.columns if str(col).isdigit()]

    # melt the data
    # XXX Need to retain other information, edit this
    melted = df.melt(
        id_vars=[
            "Date",
            " Total Max Capacity of Read Meters/KW",
            "Total Max Capacity",
            "Number of Read Meters",
            "Total Number of Meters",
        ],
        value_vars=hour_columns,
        var_name="Hour",
    )

    melted = melted.dropna()

    melted["Date"] = pd.to_datetime(melted["Date"])
    melted["Hour"] = pd.to_timedelta(melted["Hour"], unit="h")

    melted["Datetime"] = melted["Date"] + melted["Hour"]

    # Sort the DataFrame based on the 'Datetime' column
    melted_sorted = melted.sort_values(by="Datetime")
    
    melted_sorted.rename(columns={'value': 'Hourly PV Generated Units'}, inplace=True)

    return melted_sorted


hourly_data = transpose_data(hourly_data_raw)

print("--------- COMPLETE 2 --------")

--------- COMPLETE 2 --------


In [66]:
hourly_data

,Date,Total Max Capacity of Read Meters/KW,Total Max Capacity,Number of Read Meters,Total Number of Meters,Hour,Hourly PV Generated Units,Datetime
0,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 00:00:00,64.972600,2018-01-01 00:00:00
371,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 01:00:00,70.147130,2018-01-01 01:00:00
742,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 02:00:00,67.374920,2018-01-01 02:00:00
1113,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 03:00:00,65.701550,2018-01-01 03:00:00
1484,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 04:00:00,66.891940,2018-01-01 04:00:00
...,...,...,...,...,...,...,...,...
7415,2019-01-02,85111.83,129785.3,14017.0,24570.0,0 days 19:00:00,3.234269,2019-01-02 19:00:00
7786,2019-01-02,85111.83,129785.3,14017.0,24570.0,0 days 20:00:00,2.737157,2019-01-02 20:00:00
8157,2019-01-02,85111.83,129785.3,14017.0,24570.0,0 days 21:00:00,2.238523,2019-01-02 21:00:00
8528,2019-01-02,85111.83,129785.3,14017.0,24570.0,0 days 22:00:00,3.087880,2019-01-02 22:00:00


In [67]:
# 3. Save the file
hourly_data.to_csv("data/island_A/exported/hourly_data_clean.csv", index=False)

In [68]:
# 4. Apply UTC conversion
source_timezone = 'Europe/Malta'  # Replace with the desired timezone
data_hourly_utc = convert_to_utc(hourly_data, source_timezone)

In [69]:
data_hourly_utc

,Date,Total Max Capacity of Read Meters/KW,Total Max Capacity,Number of Read Meters,Total Number of Meters,Hour,Hourly PV Generated Units
datetimeUTC,,,,,,,
2017-12-31 23:00:00+00:00,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 00:00:00,64.972600
2018-01-01 00:00:00+00:00,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 01:00:00,70.147130
2018-01-01 01:00:00+00:00,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 02:00:00,67.374920
2018-01-01 02:00:00+00:00,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 03:00:00,65.701550
2018-01-01 03:00:00+00:00,2018-01-01,62654.33,112493.8,11406.0,22368.0,0 days 04:00:00,66.891940
...,...,...,...,...,...,...,...
2019-01-02 18:00:00+00:00,2019-01-02,85111.83,129785.3,14017.0,24570.0,0 days 19:00:00,3.234269
2019-01-02 19:00:00+00:00,2019-01-02,85111.83,129785.3,14017.0,24570.0,0 days 20:00:00,2.737157
2019-01-02 20:00:00+00:00,2019-01-02,85111.83,129785.3,14017.0,24570.0,0 days 21:00:00,2.238523


In [73]:
#5. Drop unused information
def drop_unused_hourly(df):
    
    df = df.drop(['Date','Hour'],1)
        
    return df

data_hourly_utc_drop = drop_unused_hourly(data_hourly_utc)

/var/folders/tk/fz0371jn0z5b2j4kj_ycxyh00000gn/T/ipykernel_24285/2225418414.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['Date','Hour'],1)


In [76]:
data_hourly_utc_drop

,Total Max Capacity of Read Meters/KW,Total Max Capacity,Number of Read Meters,Total Number of Meters,Hourly PV Generated Units
datetimeUTC,,,,,
2017-12-31 23:00:00+00:00,62654.33,112493.8,11406.0,22368.0,64.972600
2018-01-01 00:00:00+00:00,62654.33,112493.8,11406.0,22368.0,70.147130
2018-01-01 01:00:00+00:00,62654.33,112493.8,11406.0,22368.0,67.374920
2018-01-01 02:00:00+00:00,62654.33,112493.8,11406.0,22368.0,65.701550
2018-01-01 03:00:00+00:00,62654.33,112493.8,11406.0,22368.0,66.891940
...,...,...,...,...,...
2019-01-02 18:00:00+00:00,85111.83,129785.3,14017.0,24570.0,3.234269
2019-01-02 19:00:00+00:00,85111.83,129785.3,14017.0,24570.0,2.737157
2019-01-02 20:00:00+00:00,85111.83,129785.3,14017.0,24570.0,2.238523


In [79]:
# 5. Convert into Xarray and then NetCDF format
data_hourly_xarray = pdtocdf(data_hourly_utc_drop,"data_hourly_FIN")

File saved at: ./data/island_A/exported/data_hourly_FIN.nc


In [80]:
data_hourly_xarray

<xarray.Dataset>
Dimensions:                                (datetimeUTC: 8808)
Coordinates:
  * datetimeUTC                            (datetimeUTC) object 1514761200000...
Data variables:
     Total Max Capacity of Read Meters/KW  (datetimeUTC) float64 6.265e+04 .....
    Total Max Capacity                     (datetimeUTC) float64 1.125e+05 .....
    Number of Read Meters                  (datetimeUTC) float64 1.141e+04 .....
    Total Number of Meters                 (datetimeUTC) float64 2.237e+04 .....
    Hourly PV Generated Units              (datetimeUTC) float64 64.97 ... 3.556